<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/Try_DeepWalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


I got generated the graph by using `seealsology`

* https://densitydesign.github.io/strumentalia-seealsology/
* It does web crawling for any wiki-page you typed in and creates a graph for you to download.
* I used "environment protection" and "global warming" 2 wiki pages

In [0]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

## Extract RandomWalk Sequeces from the Graph

In [3]:
graph_df = pd.read_csv("environment_wiki_graph.tsv", sep = "\t")
graph_df.head()

,source,target,depth
0,environmental protection,biodiversity,1
1,environmental protection,carbon offset,1
2,environmental protection,citizen science,1
3,environmental protection,conservation biology,1
4,environmental protection,conservation movement,1


In [4]:
graph_df['source'].value_counts()

sustainable development                 38
environmentalism                        38
natural resource management             34
sustainability                          34
environmental movement                  31
ecology movement                        31
natural capital                         27
environmental protection                25
conservation movement                   24
environmental racism                    23
earth day                               23
conservation biology                    20
carbon offset                           19
anthropocene                            19
planetary boundaries                    13
public information and participation    13
participation (decision making)         13
green solutions                         13
citizen science                         13
holocene extinction                     13
renewable resource                      11
biodiversity                             9
environmental governance                 9
global cool

In [5]:
# construct the graph
G = nx.from_pandas_edgelist(graph_df, "source", "target", 
                            edge_attr=True, create_using=nx.Graph())
G

In [6]:
# The number of nodes
len(G)

367

In [0]:
def random_walk(source_node, sequence_len):
  """
  For a souce node, get its neighbours that not in the random-walk sequence,
  randomly select a neighbour as the next source node,
  repeat until reaches to the max length or there is no node to choose.

  NOTE: No need to set seed for random, cuz we want each time to generate
        a different sequence when call this function.
  """
  random_walk_sequence = [source_node]

  for i in range(sequence_len - 1):
    target_nodes = list(G.neighbors(source_node))
    target_nodes = list(set(target_nodes) - set(random_walk_sequence))
    if len(target_nodes) == 0:
      break

    selected_node = random.choice(target_nodes)
    random_walk_sequence.append(selected_node)
    source_node = selected_node

  return random_walk_sequence

In [8]:
random_walk('sustainable development', 7)

['sustainable development',
 'environmental justice',
 'environmental movement',
 'natural environment',
 'conservation movement',
 'water conservation',
 'conservation biology']

In [9]:
# get random_walk sequences for all the nodes
all_nodes = list(G.nodes)

random_walks = []
for n in tqdm(all_nodes):  # tqdm is to show the progress
  for i in range(4):
    random_walks.append(random_walk(n, 10))

print(len(random_walks))

100%|██████████| 367/367 [00:00<00:00, 11672.22it/s]

1468


In [10]:
for seq in random_walks[4:10]:
  print(seq)

['biodiversity', 'ecological indicator']
['biodiversity', 'bioversity international']
['biodiversity', 'international day for biological diversity']
['biodiversity', 'zero-force evolutionary law']
['carbon offset', 'carbon footprint', 'earth day', 'copenhagen accord']
['carbon offset', 'clean development mechanism']


## Train Skip-Gram

* [Relevant Params for Word2Vec here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

In [0]:
from gensim.models import Word2Vec

# maximum distance between current node to predicted node is 3
# negative sampling, random 10 noise words
# learning rate linearly decrease from alpha to min alpha
model = Word2Vec(window = 4, sg = 1, hs = 0,
                 negative = 10, 
                 alpha=0.03, min_alpha=0.0007, 
                 seed = 10)

model.build_vocab(random_walks, progress_per=2) # process 2 nodes before updating progress

In [17]:
# By default in Word2Vec, each node is represented by a fixed length 100
model.train(random_walks, total_examples = model.corpus_count, epochs=20, report_delay=1)

(86695, 127320)

In [19]:
import warnings
warnings.filterwarnings('ignore')

# with trained model, find similar sequence
## But input here has to be in the vocabulary
model.similar_by_word('earth day')

[("expo '74", 0.8673896789550781),
 ('world environment day', 0.8634650707244873),
 ('international mother earth day', 0.8435598611831665),
 ('earth charter', 0.8424967527389526),
 ('copenhagen accord', 0.8382109999656677),
 ('earth strike', 0.8329126834869385),
 ('pale blue dot', 0.8311952352523804),
 ('politics of global warming', 0.8281704187393188),
 ('under2 coalition', 0.8239361047744751),
 ('paris agreement', 0.8235969543457031)]

In [23]:
model.similarity('carbon footprint', 'earth day')

0.7180756